
# Семинар: классический NLP-пайплайн (spaCy) + Zipf + правила

В этом ноутбуке мы:
1) загрузим текст (Project Gutenberg: *Pride and Prejudice*);
2) сделаем базовую предобработку и посчитаем **types/tokens**;
3) построим **Zipf-графики** и посмотрим **длинный хвост** (≤ 3);
4) запустим **классический NLP-пайплайн** в `spaCy` (токенизация → POS → зависимости → NER);
5) соберём **простую систему на правилах** (rule-based intent detector) для коротких пользовательских запросов.



In [8]:
# (Colab) Установка зависимостей
!pip -q install spacy
!python -m spacy download en_core_web_sm -q


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


## 1) Загрузка данных (Gutenberg)

In [4]:

# import re
# from collections import Counter
# import requests

from pathlib import Path

# URL = "https://www.gutenberg.org/cache/epub/1342/pg1342.txt"

# raw = requests.get(URL, timeout=30).text

# Уберём служебные заголовки/хвосты Gutenberg (если маркеры есть)
# start_marker = "*** START OF THE PROJECT GUTENBERG EBOOK"
# end_marker = "*** END OF THE PROJECT GUTENBERG EBOOK"
# start_idx = raw.find(start_marker)
# end_idx = raw.find(end_marker)

# text = raw
# if start_idx != -1:
#     text = raw[start_idx:]
#     text = text.split("\n", 1)[1] if "\n" in text else text
# if end_idx != -1 and end_idx > 0:
#     text = text[:end_idx]

text = Path("Kotlovan.txt").read_text(encoding="utf-8")


print("Длина текста (символов):", len(text))
print(text[:400])


Длина текста (символов): 237873
Андрей Платонов.

                                 Котлован

                OCR: Serge Winitzki (swinitzk@hotmail.com)


    В  день  тридцатилетия  личной  жизни  Вощеву дали расчет с
небольшого механического завода, где он  добывал  средства  для
своего  существования. В увольнительном документе ему написали,
что   он   устраняется   с   производства   вследствие    роста
слабосильности в нем и


In [6]:
import re
from collections import Counter

def tokenize_rus(s: str):
    return re.findall(r"[А-Яа-яЁё]+(?:-[А-Яа-яЁё]+)*", s)

def normalize_basic(s: str, lower=True, yo_to_e=False, drop_punctuation=False):
    if yo_to_e:
        s = s.replace("ё", "е").replace("Ё", "Е")
    if lower:
        s = s.lower()
    if drop_punctuation:
        s = re.sub(r"[^А-Яа-яЁё\s]+", " ", s)
        s = re.sub(r"\s+", " ", s).strip()
    return s

def stats_to_tokens(tokens):
    count = Counter(tokens)
    N = sum(count.values())
    V = len(count)
    long_tail = sum(1 for f in count.values() if f <= 3)
    return count, N, V, long_tail, long_tail / V


versions = {
    "tokens_raw_lower": normalize_basic(text, lower=True, yo_to_e=False, drop_punctuation=False),
    "tokens_no_puctuation": normalize_basic(text, lower=True, yo_to_e=False, drop_punctuation=True),
    "tokens_no_puctuation_yo_to_e": normalize_basic(text, lower=True, yo_to_e=True, drop_punctuation=True)
}

for version_name, version_contents in versions.items():
    tokens = tokenize_rus(version_contents)
    count, N, V, long_tail, long_tail_divided = stats_to_tokens(tokens)
    print(version_name, "tokens=", N, "types=", V, "long_tail_with_types less than 4 =", long_tail, f"({long_tail_divided:.2%})")
    print("top 20:", count.most_common(20))

tokens_raw_lower tokens= 34187 types= 9808 long_tail_with_types less than 4 = 8545 (87.12%)
top 20: [('и', 1553), ('в', 879), ('не', 657), ('на', 572), ('он', 420), ('а', 419), ('что', 343), ('с', 287), ('чиклин', 284), ('его', 246), ('я', 213), ('от', 206), ('ты', 200), ('как', 199), ('но', 181), ('все', 178), ('к', 171), ('вощев', 169), ('же', 144), ('по', 143)]
tokens_no_puctuation tokens= 34412 types= 9747 long_tail_with_types less than 4 = 8482 (87.02%)
top 20: [('и', 1555), ('в', 879), ('не', 657), ('на', 572), ('он', 420), ('а', 419), ('что', 379), ('с', 287), ('чиклин', 284), ('его', 246), ('я', 216), ('как', 206), ('от', 206), ('то', 204), ('ты', 200), ('но', 181), ('все', 181), ('к', 171), ('вощев', 169), ('по', 156)]
tokens_no_puctuation_yo_to_e tokens= 34412 types= 9747 long_tail_with_types less than 4 = 8482 (87.02%)
top 20: [('и', 1555), ('в', 879), ('не', 657), ('на', 572), ('он', 420), ('а', 419), ('что', 379), ('с', 287), ('чиклин', 284), ('его', 246), ('я', 216), ('ка

## 2) Токенизация, types/tokens, частоты

In [7]:

# Простая токенизация для частот: слова из латинских букв + апострофы
tokens = re.findall(r"[A-Za-z]+(?:'[A-Za-z]+)?", text.lower())
cnt = Counter(tokens)

N_tokens = sum(cnt.values())  # tokens = все употребления
V_types = len(cnt)            # types  = уникальные слова

print(f"Tokens (всего словоупотреблений): {N_tokens:,}")
print(f"Types  (уникальных слов):        {V_types:,}")

# Топ-20
top20 = cnt.most_common(20)
top20[:10]


Tokens (всего словоупотреблений): 69
Types  (уникальных слов):        35


[('x', 30),
 ('serge', 2),
 ('winitzki', 2),
 ('the', 2),
 ('may', 2),
 ('for', 2),
 ('ocr', 1),
 ('swinitzk', 1),
 ('hotmail', 1),
 ('com', 1)]

## 3) Zipf: rank–frequency (log–log) + топ-20 + длинный хвост ≤ 3

In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Zipf данные
freqs_sorted = np.array(sorted(cnt.values(), reverse=True))
ranks = np.arange(1, len(freqs_sorted) + 1)

# 3.1 Zipf log-log
plt.figure(figsize=(7,5))
plt.loglog(ranks, freqs_sorted, marker=".", linestyle="none")
plt.title("Pride and Prejudice: закон Ципфа (ранг–частота)")
plt.xlabel("Ранг слова (log)")
plt.ylabel("Частота (log)")
plt.tight_layout()
plt.show()

# 3.2 Топ-20
df_top20 = pd.DataFrame(top20, columns=["word","count"])
plt.figure(figsize=(10,4))
plt.bar(df_top20["word"], df_top20["count"])
plt.title("Топ-20 слов по частоте")
plt.xlabel("Слово")
plt.ylabel("Частота")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

# 3.3 Длинный хвост: <= 3
tail_words = [w for w, c in cnt.items() if c <= 3]
tail_types = len(tail_words)
tail_tokens = sum(cnt[w] for w in tail_words)

print(f"Хвост (c<=3): {tail_types} типов = {tail_types/V_types:.1%} словаря")
print(f"Хвост (c<=3): {tail_tokens} токенов = {tail_tokens/N_tokens:.1%} текста")

plt.figure(figsize=(7,4))
plt.bar(["Доля ТИПОВ\n(c ≤ 3)", "Доля ТОКЕНОВ\n(c ≤ 3)"],
        [tail_types/V_types, tail_tokens/N_tokens])
plt.ylim(0, 1)
plt.title("«Длинный хвост» (c ≤ 3)")
plt.ylabel("Доля")
plt.tight_layout()
plt.show()

# 3.4 (Опционально) 10 случайных hapax (c=1)
hapax = [w for w, c in cnt.items() if c == 1]
rng = np.random.default_rng(0)
sample10 = list(rng.choice(hapax, size=10, replace=False))
sample10


## 4) Классический NLP-пайплайн в spaCy

In [ ]:

import spacy
nlp = spacy.load("en_core_web_sm")

doc = nlp("Mr. Darcy spoke politely, but Elizabeth did not change her mind about him. Then, he went to the table.")
print("SENTENCES:", [s.text for s in doc.sents])
print("TOKENS:", [t.text for t in doc])
print("POS:", [(t.text, t.pos_) for t in doc])
print("DEP:", [(t.text, t.dep_, t.head.text) for t in doc])

print("\nNER entities:")
for ent in doc.ents:
    print(ent.text, ent.label_)

for t in doc:
    print(f"{t.i:2d}  {t.text!r:12}  is_alpha={t.is_alpha}  is_punct={t.is_punct}")


### Применим пайплайн к реальному фрагменту из книги

In [ ]:

# Возьмём небольшой фрагмент (чтобы быстро работало)
snippet = " ".join(text.split()[:350])  # первые ~350 слов
doc = nlp(snippet)

# Посмотрим 15 токенов с POS/леммой
rows = []
for t in list(doc)[:15]:
    rows.append([t.text, t.lemma_, t.pos_, t.tag_])
pd.DataFrame(rows, columns=["token","lemma","POS","TAG"])


## 5) Простая rule-based система: определение интентов


Сделаем игрушечный **intent detector** для пользовательских сообщений:
- greeting / goodbye
- ask_weather
- ask_time
- math_addition
- other

Подход:
- регулярки + `spaCy Matcher`.


In [ ]:

from spacy.matcher import Matcher

matcher = Matcher(nlp.vocab)

pattern_greet = [{"LOWER": {"IN": ["hi","hello","hey","greetings"]}}]
pattern_goodbye = [{"LOWER": {"IN": ["bye","goodbye","farewell"]}}]
pattern_weather = [{"LOWER": {"IN": ["weather","rain","sunny","forecast"]}}]
pattern_time = [{"LOWER": {"IN": ["time","clock"]}}]

pattern_add_1 = [{"LOWER": "add"}, {"IS_DIGIT": True}, {"LOWER": "to"}, {"IS_DIGIT": True}]
pattern_add_2 = [{"IS_DIGIT": True}, {"TEXT": "+"}, {"IS_DIGIT": True}]

matcher.add("GREET", [pattern_greet])
matcher.add("GOODBYE", [pattern_goodbye])
matcher.add("WEATHER", [pattern_weather])
matcher.add("TIME", [pattern_time])
matcher.add("ADD", [pattern_add_1, pattern_add_2])

def detect_intent(text: str):
    doc = nlp(text)
    matches = matcher(doc)
    labels = [nlp.vocab.strings[m_id] for m_id, _, _ in matches]

    if "ADD" in labels:
        nums = [int(t.text) for t in doc if t.like_num]
        if len(nums) >= 2:
            return "math_addition", nums[0] + nums[1]
        return "math_addition", None
    if "WEATHER" in labels:
        return "ask_weather", None
    if "TIME" in labels:
        return "ask_time", None
    if "GREET" in labels:
        return "greeting", None
    if "GOODBYE" in labels:
        return "goodbye", None
    return "other", None

tests = [
    "Hello!",
    "Add 34957 to 70764",
    "What is 2 + 2 ?",
    "What's the weather tomorrow?",
    "What time is it now?",
    "Goodbye!",
    "Explain Zipf's law in one sentence."
]

for s in tests:
    intent, val = detect_intent(s)
    print(f"{s!r} -> {intent}", (f"(value={val})" if val is not None else ""))


## Домашняя мини-задача (5–10 минут)


1) Возьмите **русский роман** (любой .txt) и повторите блок **Zipf + длинный хвост**.
2) Сравните предобработки: lowercasing, удаление пунктуации, замена "ё"→"е", лемматизация (если есть инструменты).
3) Добавьте 2 новых интента и правила (Matcher/regex).


# Дополнение: русский язык — лемматизация, стемминг, POS/DEP/NER

В этом разделе:
1) покажем **лемматизацию** для русского (два варианта: `spaCy` и `pymorphy3`),
2) покажем **стемминг** (SnowballStemmer),
3) запустим **классический NLP-пайплайн** для русского: POS → синтаксические зависимости (DEP) → NER.


## 6) Установка русских инструментов

- `ru_core_news_sm` — русская модель spaCy (токены, леммы, POS, зависимости, NER)
- `pymorphy3` — сильный морфологический анализатор/лемматизатор для русского
- `nltk` — стемминг (SnowballStemmer)


In [ ]:
# (Colab) Установка для русского
!pip -q install spacy==3.7.5  nltk
!python -m spacy download ru_core_news_sm -q


## 7) Пример русского текста


In [ ]:
ru_text = (
    "Вчера Анна Каренина приехала в Москву.\n"
    "Она встретилась с братом Стивой в ресторане и долго говорила о семье и поездке.\n"
)
print(ru_text)


## 8) Лемматизация и стемминг (русский)

Сравним:
- **леммы** (нормальная форма слова) — полезно для словаря/частот;
- **стеммы** (усечённые основы) — грубее, но быстро и просто.


In [ ]:
import re

# Базовая токенизация для русского (буквы + дефис)
ru_tokens = re.findall(r"[А-Яа-яЁё]+(?:-[А-Яа-яЁё]+)?", ru_text.lower())
print("Токены:", ru_tokens)


In [ ]:
# 8.1 Стемминг (Snowball)
import nltk
from nltk.stem.snowball import SnowballStemmer

nltk.download("punkt", quiet=True)
stemmer = SnowballStemmer("russian")

ru_stems = [stemmer.stem(w) for w in ru_tokens]
print(list(zip(ru_tokens, ru_stems)))


### 8.2 Лемматизация через pymorphy2 (обычно лучше для русского)

In [ ]:
!pip -q install pymorphy3 pymorphy3-dicts-ru

In [ ]:
import pymorphy3
morph = pymorphy3.MorphAnalyzer()

ru_lemmas_pym = [morph.parse(w)[0].normal_form for w in ru_tokens]
print(list(zip(ru_tokens, ru_lemmas_pym)))


### 8.3 Лемматизация через spaCy (удобно, когда вы и так используете пайплайн)

In [ ]:
import spacy
from spacy import displacy

nlp_ru = spacy.load("ru_core_news_sm")

doc_ru = nlp_ru(ru_text)
print([(t.text, t.lemma_, t.pos_) for t in doc_ru if t.is_alpha])

displacy.render(doc_ru, style="dep", jupyter=True, options={"distance": 90})


## 9) POS-теги + синтаксические зависимости (DEP) + NER для русского

In [ ]:
import pandas as pd

# 9.1 POS + морфология
rows = []
for t in doc_ru:
    if t.is_space:
        continue
    rows.append([t.text, t.lemma_, t.pos_, t.tag_, t.morph.to_json()])
pd.DataFrame(rows, columns=["token","lemma","POS","TAG","morph"])


In [ ]:
# 9.2 Синтаксические зависимости (DEP): token -> head
dep_rows = []
for t in doc_ru:
    if t.is_space:
        continue
    dep_rows.append([t.text, t.dep_, t.head.text])
pd.DataFrame(dep_rows, columns=["token","dep","head"])


In [ ]:
# 9.3 NER (именованные сущности)
[(ent.text, ent.label_) for ent in doc_ru.ents]


## Матчеры на основе тэгов

In [ ]:
from spacy.matcher import Matcher

matcher = Matcher(nlp.vocab)

pattern_superl_noun = [
    {"POS": "DET", "OP": "?"},  # optional: the/a
    {"POS": "ADJ", "TAG": {"IN": ["JJS"]}},  # superlative adj: best, worst
    {"POS": "NOUN"}
]
matcher.add("SUPERL_NOUN", [pattern_superl_noun])

doc = nlp("This is the best book and the worst idea.")
for mid, start, end in matcher(doc):
    print(nlp.vocab.strings[mid], doc[start:end].text)


In [ ]:
pattern_past_verb_object = [
    {"POS": "VERB", "TAG": "VBD"},            # past tense verb
    {"POS": "DET", "OP": "?"},                # optional determiner
    {"POS": {"IN": ["NOUN", "PROPN"]}}        # object-ish noun
]
matcher.add("PAST_VERB_OBJ", [pattern_past_verb_object])

doc = nlp("She visited London and bought a car yesterday.")
for mid, start, end in matcher(doc):
    print(nlp.vocab.strings[mid], doc[start:end].text)


In [ ]:
pattern_np = [
    {"POS": "DET", "OP": "?"},
    {"POS": "ADJ", "OP": "*"},
    {"POS": {"IN": ["NOUN", "PROPN"]}}
]
matcher.add("NP_RULE", [pattern_np])

doc = nlp("I saw a very old house and Mr. Darcy.")
for mid, start, end in matcher(doc):
    print(doc[start:end].text)


In [ ]:
pattern_question = [
    {"POS": "AUX"},
    {"POS": "PRON"},
    {"POS": "VERB"}
]
matcher.add("QUESTION_AUX_PRON_VERB", [pattern_question])

doc = nlp("Do you know Zipf's law? Can you help me?")
for mid, start, end in matcher(doc):
    print(nlp.vocab.strings[mid], doc[start:end].text)


In [ ]:
from spacy.matcher import Matcher
matcher = Matcher(nlp_ru.vocab)

pattern_adj_noun = [
    {"POS": "ADJ"},
    {"POS": "NOUN"}
]
matcher.add("ADJ_NOUN", [pattern_adj_noun])

doc = nlp_ru("Старый дом стоял на тихой улице.")
for mid, start, end in matcher(doc):
    print(nlp_ru.vocab.strings[mid], doc[start:end].text)


## 10) Мини-задача для семинара (русский)

1) Возьмите фрагмент русского романа (например, 30–100k символов).
2) Сравните частоты **по токенам**, **по стеммам**, **по леммам**:
   - Как меняются топ-20?
   - Как меняется доля “длинного хвоста” (≤3) по **types**?
3) Выберите 5 предложений и сравните:
   - POS/DEP/NER от `spaCy`
   - леммы от `pymorphy3`
4) Составьте список имен персонажей
5) Найдите как можно больше предложений, где здороваются
